In [12]:
#!pip3 install "psycopg[binary]"

In [32]:
import config
import psycopg
import pandas as pd
from psycopg.rows import dict_row

In [33]:
def pgDictToConn(secretDict):
    pgStrs = []
    for key in secretDict:
        pgStrs.append('{}={}'.format(key, secretDict[key]))
    return ' '.join(pgStrs)


In [34]:
#Init
pgConnStr = pgDictToConn(config.pgSecrets)

In [24]:
def getTicker(symbol):
    with psycopg.connect(pgConnStr) as conn:
        with conn.cursor(row_factory=dict_row) as cur:
            stmt = '''SELECT * FROM yfinance_daily WHERE ticker = %s'''
            data = (symbol, )
            result = cur.execute(stmt, data).fetchall()
            desc = cur.description
            cols = [col[0] for col in desc]
            conn.commit()
    return pd.DataFrame(data=result, columns=cols)

In [43]:
def getTickerList():
    with psycopg.connect(pgConnStr) as conn:
        with conn.cursor(row_factory=dict_row) as cur:
            stmt = '''SELECT DISTINCT ticker FROM yfinance_daily ORDER BY ticker'''
            result = cur.execute(stmt).fetchall()
            conn.commit()
            return [row['ticker'] for row in result]

In [ ]:
def getAllData():
    with psycopg.connect(pgConnStr) as conn:
        with conn.cursor(row_factory=dict_row) as cur:
            stmt = '''SELECT * FROM yfinance_daily ORDER By ticker, datetime'''
            result = cur.execute(stmt).fetchall()
            desc = cur.description
            cols = [col[0] for col in desc]
            conn.commit()
    return pd.DataFrame(data=result, columns=cols)

In [46]:
#Get List of all tickers
getTickerList()


['AAPL', 'AMZN', 'GOOG', 'GOOGL', 'MSFT']

In [45]:
#Get Specific Ticker
getTicker('GOOG').sample(4)

,id,ticker,datetime,high,low,close,vol,dividend,split,gain,open
98,348,GOOG,2023-01-26,99.209999,96.820000,99.160004,24542100,0.0,0.0,nan,98.279999
114,364,GOOG,2023-02-17,95.750000,93.449997,94.589996,31074100,0.0,0.0,nan,95.070000
37,287,GOOG,2022-10-27,95.169998,91.900002,92.599998,54036500,0.0,0.0,nan,94.309998
102,352,GOOG,2023-02-01,102.190002,98.419998,101.430000,26392600,0.0,0.0,nan,99.739998
